### This jupyter notebook is to match face of a given photo with all the faces from photos folder 

In [2]:
# importing libraries
import cv2
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

c:\Users\sethi\anaconda3\envs\myenvpytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('test_images_aligned') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list
        


### Saving data into data.pt file

In [4]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file

### Matching face id of the given photo with available data from data.pt file

In [29]:

def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    img1 = np.array(img)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    
    
    
    boxes, _ = mtcnn.detect(img)
    if prob>0.90:
        
        box = [int(x) for x in boxes[0]]
        original_img = img1.copy() # storing copy of frame before drawing on it
        min_dist = round(min(dist_list),2)
        name = name_list[idx_min]
        
        if min_dist<0.90:
            frame = cv2.putText(img1, name+' '+str(min_dist), (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0),5, cv2.LINE_AA)
        else:
            frame = cv2.putText(img1, "Unknown", (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0),5, cv2.LINE_AA)
        frame = cv2.rectangle(img1, (box[0],box[1]) , (box[2],box[3]), (255,0,0), 5)
        

    return (name_list[idx_min], min(dist_list), frame)


result = face_match('photos/bradley_cooper/1.jpg', 'data.pt')
print(result[2].shape)
cv2.imwrite(f"{result[0]}.jpg",cv2.cvtColor(result[2], cv2.COLOR_BGR2RGB))


print('Face matched with: ',result[0], 'With distance: ',result[1])

(3600, 2341, 3)
Face matched with:  bradley_cooper With distance:  0.20242971181869507
